## 7.3 AppWidget - podstawy

W tej aplikacji zobaczymy jak utworzyć prosty widget. Dodamy widget w formie przycisku na ekran główny urządzenia i dodamy do niego ekran konfiguracji. Wszystkie elementy w tym przykładzie dodamy ręcznie, szybszym rozwiązaniem jest dodanie widgetu wprost przez wybranie z menu kontekstowego **New -> Widget -> App Widget**.

W pierwszym kroku dodamy layout naszego widgeta - standardowo do katalogu `layout` dodajemy plik `app_widget_layout`. Layout widgetu posiada pewne ograniczenia w dostępnych elementach.

```xml
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="match_parent">

    <Button
        android:id="@+id/widgetButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_centerInParent="true"
        android:text="Widget"/>

</RelativeLayout>
```

Wpierw otworzymy okno naszej aplikacji po naciśnięciu tego przycisku, następnie dodamy ekran konfiguracyjny i zmienimy tekst przy dodaniu widgetu na ekran.

Drugim krokiem będzie implementacja klasy providera, która zawiera implementacje metod reagujących na zmianęv stanu naszego widgetu lub na aktualizację danych.

In [ ]:
public class WidgetProvider extends AppWidgetProvider {}

W tym przykładzie będzie interesować nas tylko implementacja metody `onUpdate`

In [ ]:
@Override
public void onUpdate(Context context, AppWidgetManager appWidgetManager, int[] appWidgetIds) {
    super.onUpdate(context, appWidgetManager, appWidgetIds);
}

Metoda przyjmuje trzy argumenty
- `context`
- `AppWidggetManager` - ponieważ widget działa na innym procesie będziemy się komunikować przez odpowedni manager
- `appWidgetIds` - ponieważ możemy mieć wiele instancji widgetu na ekranie domowym potrzebujemy sposób na nadanie unikalnych identyfikatorów

Standardowo przechodzi się przez wszystkie identyfikatory dostępne w tabeli

In [ ]:
    for (int appWidgetId : appWidgetIds){}

Chcemy otworzyć ekran główny naszej aplikacji, więc dodamy tutaj odpowiedni `Intent` oraz (tak jak w powiadomieniach) `PendingIntent`

In [ ]:
    Intent intent = new Intent(context, MainActivity.class);
    PendingIntent pendingIntent = PendingIntent.getActivity(
            context,
            0,
            intent,
            PendingIntent.FLAG_IMMUTABLE);

Nastęnie layout naszego widgetu będziemy przekazywać jako `RemoteView`

In [ ]:
    RemoteViews views = new RemoteViews(
                    context.getPackageName(),
                    R.layout.app_widget_layout);

Następnie chcemy dodać obsługę `onClick` przycisku w layoucie widgetu, w tym celu musimy wywołać metodę `setOnClickPendingIntent`

In [ ]:
    views.setOnClickPendingIntent(R.id.widgetButton, pendingIntent);

Następnie wywołujemy `updateAppWidget` na każdej instancji naszego widgetu

In [ ]:
    appWidgetManager.updateAppWidget(appWidgetId, views);

Pełna implementacja metody `onUpdate`

In [ ]:
@Override
public void onUpdate(Context context, AppWidgetManager appWidgetManager, int[] appWidgetIds) {
    super.onUpdate(context, appWidgetManager, appWidgetIds);
    for (int appWidgetId : appWidgetIds){
        Intent intent = new Intent(context, MainActivity.class);
        PendingIntent pendingIntent = PendingIntent.getActivity(
                context,
                0,
                intent,
                PendingIntent.FLAG_IMMUTABLE);

        RemoteViews views =
                new RemoteViews(
                        context.getPackageName(),
                        R.layout.app_widget_layout);

        views.setOnClickPendingIntent(R.id.widgetButton, pendingIntent);
        appWidgetManager.updateAppWidget(appWidgetId, views);
    }
}

Kolejnym krokiem będzie zdefiniowanie rozmiarów konteneru na widget i kilku dodatkowych informacji, robimy to w katalogu `xml`. Dodajmy nowy plik `app_widget_info.xml`, jako **root element** wybieramy `appwidget-provider`

```xml
<?xml version="1.0" encoding="utf-8"?>
<appwidget-provider xmlns:android="http://schemas.android.com/apk/res/android"
    >

</appwidget-provider>
```

dodamy kilka elementów
- layout startowy - będzie to layout zdefiniowany dla naszego widgetu
```xml
    android:initialLayout="@layout/app_widget_layout"
```


- rozmiar minimalny - podajemy `40dp` oraz `110dp`, czyli nasz rozmiar będzie wynosił 1 x 2 komórki
```xml
    android:minHeight="40dp"
    android:minWidth="110dp"
```


- umożliwimy zmniejszenie rozmiarów do 1 kolumny
```xml
    android:minResizeWidth="40dp"
```


- umożliwimy zmianę rozmiaru widgetu w obu kierunkach
```xml
    android:resizeMode="vertical|horizontal"
```


- aktualizacja widgetu zazwyczaj nie mniej niż 30 minut
```xml
    android:updatePeriodMillis="36000000"
```


- widget będzie dostępny tylko na ekranie domowym
```xml
    android:widgetCategory="home_screen"
```


pełny plik `app_widget_info.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<appwidget-provider xmlns:android="http://schemas.android.com/apk/res/android"
   android:initialLayout="@layout/app_widget_layout"
    android:minHeight="40dp"
    android:minWidth="110dp"
    android:minResizeWidth="40dp"
    android:resizeMode="vertical|horizontal"
    android:updatePeriodMillis="36000000"
    android:widgetCategory="home_screen">

</appwidget-provider>
```

W kolejnym kroku musimy zarejestrować nasz widget w manifeście aplikacji - przechodzimy do `AndroidManifest`. w tagach `receiver` dodajemy nazwę naszego widgetu (wewnątrz `application`)


```xml
<receiver android:name=".WidgetProvider"
    android:exported="false">
</receiver>
```

Następnie dodajemy `intent-filter` i definiujemy dostępne akcje
```xml
<receiver android:name=".WidgetProvider"
    android:exported="false">
    <intent-filter>
        <action android:name="android.appwidget.action.APPWIDGET_UPDATE"/>
    </intent-filter>
</receiver>
```

Ostatnię wymaganą informacją jest wskazanie providera i konfiguracji

```xml
<receiver android:name=".WidgetProvider"
    android:exported="false">
    <intent-filter>
        <action android:name="android.appwidget.action.APPWIDGET_UPDATE"/>
    </intent-filter>
    <meta-data
        android:name="android.appwidget.provider"
        android:resource="@xml/app_widget_info"/>
</receiver>
```

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/SMYlTXKvTUc8W5gIvZ/giphy.gif?cid=790b761136c8abe5f6d73470d6cef5ccce8c5ab1a89cebcc&rid=giphy.gif&ct=g" width="150" />

W kolejnym kroku dodamy aktywność umożliwiającą konfigurowanie naszego widgetu, rozpocznijmy od layoutu

```xml
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".WidgetConfigActivity"
    android:orientation="vertical">

    <EditText
        android:id="@+id/widgetConfigEditText"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:hint="set button text"
        android:layout_margin="16dp"/>

    <Button
        android:id="@+id/widgetConfigButton"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:text="confirm"/>

</LinearLayout>
```

Przejdźmy do `WidgetConfigActivity` i dodajmy `ViewBinding`

In [ ]:
public class WidgetConfigActivity extends AppCompatActivity {

    private ActivityWidgetConfigBinding binding;
    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivityWidgetConfigBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());
    }
}

Naszą konfigurację będziemy przechowywać w `SharedPreferences`. Wstępnie będziemy potrzebować dwa klucze dla `SharedPreferences` oraz klucz umożliwiający weryfikację poprawności przesłania danych do widgetu

In [ ]:
public static final String SHARED_PREFS = "shared_prefs";
public static final String KEY_BUTTON_TEXT = "keyButton";

Następnie dodamy identyfikator naszego widgetu

In [ ]:
private int appWidgetId = AppWidgetManager.INVALID_APPWIDGET_ID;

Ekran konfiguracji zostanie uruchomiony w momencie umieszczenia widgetu na ekranie domowym - gdy to się stanie do aktywności zostanie przekazanych `Intent`, więc musimy go odebrać

In [ ]:
Intent configIntent = getIntent();

Następnie musimy rozpakować dane

In [ ]:
Bundle extras = configIntent.getExtras();

Z otrzymanych `extras` wyciągamy `id` widgetu

In [ ]:
if (extras != null){
    appWidgetId = extras.getInt(AppWidgetManager.EXTRA_APPWIDGET_ID,
            AppWidgetManager.INVALID_APPWIDGET_ID);
}

Jeżeli otrzymane `id` jest niepoprawne i równe wartości domyślnej wyciągniętej z `extras` (`INVALID_APPWIDGET_ID`) chcemy zakończyć działanie aktywności

In [ ]:
if (appWidgetId == AppWidgetManager.INVALID_APPWIDGET_ID)
    finish();

Następnie implementujemy `onClick` przycisku. Dalej chcemy otwierać aktywność główną, więc implementujemy dokładnie tak jak poprzednio

In [ ]:
binding.widgetConfigButton.setOnClickListener(v -> {
    AppWidgetManager appWidgetManager = AppWidgetManager.getInstance(this);
    Intent intent = new Intent(this, MainActivity.class);
    PendingIntent pendingIntent = PendingIntent.getActivity(
            this,
            0,
            intent,
            PendingIntent.FLAG_IMMUTABLE);

Następnie wyciągamy tekst z pola `EditText` i jeżeli nie jest pusty wykonujemy kilka czynności. Wpierw tworzymy `RemoteViews`, który da nam dostęp do layoutu widgetu. Następnie ustawiamy `onClick` przycisku na widgecie

In [ ]:
if (!text.isEmpty()){
    RemoteViews views = new RemoteViews(
            WidgetConfigActivity.this.getPackageName(),
            R.layout.app_widget_layout
    );

    views.setOnClickPendingIntent(R.id.widgetButton, pendingIntent);

Następnie chcemy ustawić tekst przycisku, robimy to wywołując metodę `setCharSequence`, przyjmuje trzy argumenty
- id elementu który chcemy zmodyfikować
- nazwę metody którą chcemy wywołać (podajemy jako `String`) - tutaj będzie to `setText`
- `String` który chcemy przkekazać

In [ ]:
    views.setCharSequence(R.id.widgetButton, "setText", text);

Następnie wywołujemy `updateAppWidget`

In [ ]:
    appWidgetManager.updateAppWidget(appWidgetId, views);

Samą wartość będziemy przechowywać w `SharedPreferences`

In [ ]:
    SharedPreferences sharedPreferences = 
        getSharedPreferences(SHARED_PREFS, MODE_PRIVATE);
    SharedPreferences.Editor editor = sharedPreferences.edit();
    editor.putString(KEY_BUTTON_TEXT + appWidgetId, text);
    editor.apply();

Ostatnim krokiem jest stworzenie i przekazanie `Intent` z identyfikatorem widgetu który aktualizujemy

In [ ]:
    Intent resultIntent = new Intent();
    resultIntent.putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId);
    setResult(RESULT_OK, resultIntent);

Na koniec kończymy działanie aktynwości konfiguracyjnej

In [ ]:
    finish();

W kolejnym kroku musimy ppoinformować o przeznaczeniu aktywności w `AndroidManifest`, czyli wskazujemy że `WidgetConfigActivity` odpowiada za konfigurację widgetu

```xml
<activity
    android:name=".WidgetConfigActivity"
    android:exported="false">
    <intent-filter>
        <action android:name="android.appwidget.action.APPWIDGET_CONFIGURE"/>
    </intent-filter>
</activity>
```

W pliku `app_widget_info.xml` wskazujemy aktywność odpowiedzialną za konfigurację

```xml
android:configure="pl.edu.uwr.pum.widgetbasicsjava.WidgetConfigActivity"
```

Musimy jeszcze wprowadzić zmiany w klasie `WidgetProvider`, wyciągniemy tekst z `SharedPreferences` i uaktualnimy text przycisku.

In [ ]:
@Override
public void onUpdate(Context context, AppWidgetManager appWidgetManager, int[] appWidgetIds) {
    super.onUpdate(context, appWidgetManager, appWidgetIds);
    for (int appWidgetId : appWidgetIds){
        Intent intent = new Intent(context, MainActivity.class);
        PendingIntent pendingIntent = PendingIntent.getActivity(
                context,
                0,
                intent,
                PendingIntent.FLAG_IMMUTABLE);

        SharedPreferences preferences = context.getSharedPreferences(
            WidgetConfigActivity.SHARED_PREFS, 
            Context.MODE_PRIVATE);
        String text = preferences.
            getString(WidgetConfigActivity.KEY_BUTTON_TEXT + appWidgetId, "Click me");

        RemoteViews views =
                new RemoteViews(
                        context.getPackageName(),
                        R.layout.app_widget_layout);

        views.setOnClickPendingIntent(R.id.widgetButton, pendingIntent);
        views.setCharSequence(R.id.widgetButton, "setText", text);
        appWidgetManager.updateAppWidget(appWidgetId, views);
    }
}

Możemy przetestować aplikację

<img src="https://media0.giphy.com/media/eOVuYgGRlI3wZBfE1g/giphy.gif?cid=790b7611e2b3d01b76b1ccf0996ad803a5ecdfe4030108a5&rid=giphy.gif&ct=g" width="150" />